### comparing xgboost model with 7 features vs all 985 features

In [1]:
import sys
sys.path.append('..')

In [2]:
from core.dataset import *
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

#### identifying all feature columns corresponding to 7 selected features

0. max_value
1. power
2. ipt
3. numpeaks
4. valley_amps
5. valley_times
6. integrals_pos
7. trough2peak

##### Previously: ~ 62% accuracy when trained with all features

In [3]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [4]:
dataset = Data(load_dir, ligands, sheet_type, merge=True, numpy=False)

##### Extract the 7 seven features from this dataset

In [5]:
dataset.columns

Index([  'time_series_1',   'time_series_2',   'time_series_3',
         'time_series_4',   'time_series_5',   'time_series_6',
         'time_series_7',   'time_series_8',   'time_series_9',
        'time_series_10',
       ...
        'fold_change_92',  'fold_change_93',  'fold_change_94',
        'fold_change_95',  'fold_change_96',  'fold_change_97',
        'fold_change_98', 'max_fold_change',       'max_value',
                       0],
      dtype='object', length=985)

In [6]:
extracted_features = [dataset['integrals_pos_38'], dataset['min_trough2peak'], dataset['integrals_pos_11'], dataset['integrals_40'],
                      dataset['integrals_pos_14'], dataset['time_series_2'], dataset['integrals_pos_12']]

In [7]:
data = pd.DataFrame()

In [8]:
data = pd.concat(extracted_features, axis=1).to_numpy()
labels = dataset.iloc[:, [984]].to_numpy()
labels = np.reshape(labels, (-1, ))

In [13]:
labels.shape

(14376,)

In [14]:
X = np.hstack([data, labels.T])

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [ ]:
X.shape

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.1, random_state=42)

#### training the model on the top 7 features

In [ ]:
model = xgb.XGBClassifier()

In [ ]:
model.fit(X_train, y_train)

In [ ]:
pred = model.predict(X_val)

In [ ]:
pred

In [ ]:
acc = 0
for i, j in zip(pred, y_val):
    if i == j:
        acc += 1
print(f'Accuracy: {acc / len(y_val)}')

#### Accuracy went from 62% -> 48% when trained on 7 features vs 985

In [ ]:
### ignore everything below this line for now

In [ ]:
# print(dataset.columns.get_loc('power_1'), dataset.columns.get_loc('power_125'))
# power = dataset.iloc[:, [i for i in range(650, 775)]]
# print(dataset.columns.get_loc('max_value'))
# max_value = dataset.iloc[:, [983]]
# print(dataset.columns.get_loc('ipt_1'), dataset.columns.get_loc('ipt_15'))
# ipt = dataset.iloc[:, [i for i in range(449, 464)]]
# print(dataset.columns.get_loc('num_peaks'))
# num_peaks = dataset.iloc[:, [464]]
# print(dataset.columns.get_loc('valley_amps_1'), )